In [ ]:
''' 
Лабораторная работа №5: Реализация голосового ввода для управления функциями информационных систем
1. Необходимо реализовать модуль, который записывает команду голосом с микрофона по умолчанию, превращает ее в текст, извлекает из текста
ключевые слова, если находит в ключевых словах команду, выполняет ее.
2. Для этого нужно создать виртуальную среду python -m venv ./venv
3. Установить библиотеку для записи аудио pyminiaudio (GitHub-irmen/pyminiaudio: python interface to the miniaudio audio playback, recording,decoding and conversion library) 
или pyaudio (PyAudio: Cross-platform audio I/O for Python, with PortAudio)
4. Установить платформу для распознавания речи Whisper (GitHub-openai/whisper: Robust Speech Recognition via Large-Scale Weak Supervision):
pip install git+https://github.com/openai/whisper.git
5. Реализовать препроцессинг голосовой команды и извлечение ключевых слов для вариантов ниже:
 1. для генерации ссылки на поиск, например, в Яндексе произвольного текста в браузере по умолчанию
 2. для открытия видеофайла из файловой системы в плеере по умолчанию или поиска видео на каком-либе видеосервисе
 3. для запуска любого приложения для коммуникации (ВК, Телеграм и т.п.) и ввода сообщения в окне мессенджера
6. Для этого каждую команду необходимо предварительно обработать: токенизировать (изучите NLTK и конкретно nltk.tokenize ) и нормализовать
(лемматизировать) (через pymorphy3).
7. Реализовать через subprocess или иные средства выполнение указанных команд
8. Установить ollama: Ollama, выбрать подходящие локальные модели (deepseek-r1, qwq, Qwen-2.5-coder и т.д.)
9. Реализовать взаимодействие с моделью через голосовой ввод и Ollama API.
10. Реализовать на языке C++ алгоритмы сортировки пузырьком, выбором, вставками, подсчетом, блочную сортировку и сортировку простым слиянием
посредством ввода голосовых инструкций LLM (возможные ошибки тоже корректировать через инструктирование модели).
'''

In [8]:
import os
import pyaudio
import wave
import whisper
import nltk
from nltk.tokenize import word_tokenize
from pymorphy3 import MorphAnalyzer
import subprocess
import os
import pyautogui
import time


# Загрузите модель Whisper
model = whisper.load_model("base")  # Выберите нужную модель

# Загрузите NLTK ресурсы
nltk.download('punkt')

# Инициализация морфологического анализатора
morph = MorphAnalyzer()

# Функция для записи аудио с микрофона
def record_audio(filename):
    chunk = 1024  # Размер блока
    format = pyaudio.paInt16  # Формат аудио
    channels = 1  # Количество каналов
    rate = 44100  # Частота дискретизации
    record_seconds = 5  # Время записи в секундах
    p = pyaudio.PyAudio()
    stream = p.open(format=format, channels=channels,
                    rate=rate, input=True,
                    frames_per_buffer=chunk)
    print("Запись... (время записи 5 секунд)")
    frames = []
    for _ in range(0, int(rate / chunk * record_seconds)):
        data = stream.read(chunk)
        frames.append(data)
    print("Запись завершена.")
    stream.stop_stream()
    stream.close()
    p.terminate()
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))
    # Проверка, создан ли файл
    if os.path.exists(filename):
        print(f"Файл {filename} успешно создан.")
    else:
        print(f"Не удалось создать файл {filename}.")

def transcribe_audio(filename):
    result = model.transcribe(filename)
    return result['text']

def process_command(command):
    # Токенизация и лемматизация
    tokens = word_tokenize(command.lower())
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]
    if "поиск" in lemmatized_tokens:
        query = ' '.join(lemmatized_tokens[lemmatized_tokens.index("поиск")+1:])
        search_yandex(query)
    elif "видео" in lemmatized_tokens:
        open_video(lemmatized_tokens)
    elif "телеграм" in lemmatized_tokens:
        send_message(lemmatized_tokens)
    else:
        print("Неизвестная команда.")

# Поиск в интернете
def search_yandex(query):
    url = f"https://yandex.ru/search/?text={query}"
    print(f"Открытие ссылки: {url}")
    subprocess.run(["start", "browser", url], shell=True)  # Для Windows

# Открытие видео
def open_video(tokens):
    # Найти индекс слова "видео"
    try:
        video_idx = tokens.index("видео")
    except ValueError:
        print("Команда для видео некорректна.")
        return
    # Собрать название файла из слов после "видео"
    filename_tokens = tokens[video_idx + 1:]
    if not filename_tokens:
        print("Не указано название файла после слова 'видео'.")
        return
    filename = ' '.join(filename_tokens)
    # Добавьте расширение, если его нет
    if not filename.lower().endswith('.mpeg'):
        filename += '.mpeg'
    if os.path.exists(filename):
        print(f"Открытие видеофайла: {filename}")
        subprocess.run(["start", filename], shell=True)  # Для Windows
    else:
        print(f"Файл не найден: {filename}")

# Открытие телеграмма
def send_message(tokens):
    message = ' '.join(tokens[tokens.index("телеграм") + 1:])  # Изменено на "телеграм" 
    if not message:
        print("Не указано сообщение для отправки.")
        return
    # Открытие Telegram (если он еще не открыт)
    print(f"Открытие телеграмма: {message}")
    subprocess.run(["start", "Telegram"], shell=True)  # Убедитесь, что путь к Telegram корректен
    time.sleep(5)  # Даем время на загрузку приложения
    # Поиск нужного чата (например, по имени контакта)
    pyautogui.hotkey('ctrl', 'f')  # Открытие поиска
    time.sleep(1)  # Небольшая пауза
    pyautogui.typewrite("Имя контакта или группы")  # Замените на имя нужного контакта
    time.sleep(1)  # Небольшая пауза
    # Выбор контакта из списка
    pyautogui.press('enter')
    time.sleep(1)  # Небольшая пауза
    # Ввод сообщения
    pyautogui.typewrite(message)
    time.sleep(1)  # Небольшая пауза
    # Отправка сообщения
    #pyautogui.press('enter')
    #print(f"Отправка сообщения: {message}")

def main():
    audio_file = "command.wav"
    record_audio(audio_file)
    model = whisper.load_model("base")
    command_text = transcribe_audio(audio_file)
    result = model.transcribe(audio_file)
    print (result)
    process_command(command_text)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sergei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Запись... (время записи 5 секунд)
Запись завершена.
Файл command.wav успешно создан.
{'text': '', 'segments': [], 'language': 'nn'}
Неизвестная команда.


In [4]:
# Помск в интернете
audio_file = "command.wav"
record_audio(audio_file)
model = whisper.load_model("base")
command_text = transcribe_audio(audio_file)
result = model.transcribe(audio_file)
print (result)
process_command(command_text)

Запись... (время записи 5 секунд)
Запись завершена.
Файл command.wav успешно создан.
{'text': ' поиск 1 тигр 2 3', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 6.4, 'text': ' поиск 1 тигр 2 3', 'tokens': [50364, 2801, 3323, 755, 502, 1069, 35600, 568, 805, 50684], 'temperature': 0.0, 'avg_logprob': -0.7692571119828657, 'compression_ratio': 0.6944444444444444, 'no_speech_prob': 0.10174372047185898}], 'language': 'ru'}
Открытие ссылки: https://yandex.ru/search/?text=1 тигр 2 3


In [6]:
# Открытие видео
audio_file = "command.wav"
record_audio(audio_file)
model = whisper.load_model("base")
command_text = transcribe_audio(audio_file)
result = model.transcribe(audio_file)
print (result)
process_command(command_text)

Запись... (время записи 5 секунд)
Запись завершена.
Файл command.wav успешно создан.
{'text': ' видео тигр', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.7600000000000002, 'text': ' видео тигр', 'tokens': [50364, 15589, 1069, 35600, 50552], 'temperature': 0.0, 'avg_logprob': -0.9439678986867269, 'compression_ratio': 0.7037037037037037, 'no_speech_prob': 0.18473391234874725}], 'language': 'ru'}
Открытие видеофайла: тигр.mpeg


In [12]:
# Открытие телеграмма
audio_file = "command.wav"
record_audio(audio_file)
model = whisper.load_model("base")
command_text = transcribe_audio(audio_file)
result = model.transcribe(audio_file)
print (result)
process_command(command_text)

Запись... (время записи 5 секунд)
Запись завершена.
Файл command.wav успешно создан.
{'text': ' Телеграм 1,2,3', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 4.0, 'text': ' Телеграм 1,2,3', 'tokens': [50364, 3200, 1414, 4953, 15837, 502, 11, 17, 11, 18, 50564], 'temperature': 0.0, 'avg_logprob': -0.5748924414316813, 'compression_ratio': 0.7333333333333333, 'no_speech_prob': 0.2935825288295746}], 'language': 'ru'}
Открытие телеграмма: 1,2,3


In [8]:
# Нераспознанная команда
audio_file = "command.wav"
record_audio(audio_file)
model = whisper.load_model("base")
command_text = transcribe_audio(audio_file)
result = model.transcribe(audio_file)
print (result)
process_command(command_text)

Запись... (время записи 5 секунд)
Запись завершена.
Файл command.wav успешно создан.
{'text': ' 1-2 работа', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 4.0, 'text': ' 1-2 работа', 'tokens': [50364, 502, 12, 17, 9197, 386, 50564], 'temperature': 0.0, 'avg_logprob': -0.6208469867706299, 'compression_ratio': 0.64, 'no_speech_prob': 0.10215211659669876}], 'language': 'ru'}
Неизвестная команда.
